In [67]:
# === Config ===
URL = "https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=3"
PERIODO = "12"
ANIO    = "2024"
HEADLESS = True

import re, nest_asyncio
from bs4 import BeautifulSoup
from IPython.display import HTML, display
from playwright.async_api import async_playwright

nest_asyncio.apply()

async def fetch_html(url: str, periodo=PERIODO, anio=ANIO, headless=HEADLESS) -> str:
    async with async_playwright() as p:
        br = await p.chromium.launch(headless=headless)
        ctx = await br.new_context(locale="es-CL")
        page = await ctx.new_page()
        page.set_default_timeout(45000)
        await page.goto(url, wait_until="domcontentloaded")

        # Periodo/Año + Consultar (si existen)
        try:
            form = page.locator("form").first
            sels = form.locator("select")
            if await sels.count() >= 1:
                await sels.nth(0).select_option(value=str(periodo))
            if await sels.count() >= 2:
                try:
                    await sels.nth(1).select_option(value=str(anio))
                except:
                    await sels.nth(1).select_option(label=str(anio))
            try:
                await form.get_by_role("button", name="Consultar").click()
            except:
                try:
                    await page.locator("button:has-text('Consultar')").click()
                except:
                    await page.locator("input[type=submit']").first.click()
        except Exception:
            pass

        # Espera a que la carátula exista y tenga filas
        await page.wait_for_selector("#caratula table", timeout=30000)
        await page.wait_for_function("document.querySelectorAll('#caratula table tr').length > 5", timeout=30000)

        html = await page.content()
        await br.close()
        return html

def extraer_num_empleados(html: str):
    soup = BeautifulSoup(html, "lxml")

    # Vamos directo a la carátula
    caratula = soup.select_one("div#caratula table")
    if not caratula:
        return None, None   # (fila_html, valor)

    # Recorremos las filas; primera celda debe ser 1.15.00.00
    for tr in caratula.find_all("tr"):
        tds = tr.find_all("td", recursive=True)
        if not tds:
            continue
        first = re.sub(r"\s+", " ", tds[0].get_text(" ", strip=True))
        if first == "1.15.00.00":
            # valor = último <td> (conservamos formato tal cual)
            valor = tds[-1].get_text(" ", strip=True)
            # para mostrar la fila tal cual:
            fila_html = f"<table style='border-collapse:collapse;font-family:system-ui'><tr>" + \
                        "".join(f"<td style='border:1px solid #ccc;padding:6px 8px'>{td.get_text(' ', strip=True)}</td>" for td in tds) + \
                        "</tr></table>"
            return fila_html, valor

    # Fallback: buscar por texto "N° de Empleados" y subir al <tr>
    nodo = caratula.find(string=re.compile(r"N[\u00B0º]?\s*de\s*Empleados", re.I))
    if nodo:
        tr = nodo.find_parent("tr")
        if tr:
            tds = tr.find_all("td", recursive=True)
            valor = tds[-1].get_text(" ", strip=True) if tds else None
            fila_html = f"<table style='border-collapse:collapse;font-family:system-ui'><tr>" + \
                        "".join(f"<td style='border:1px solid #ccc;padding:6px 8px'>{td.get_text(' ', strip=True)}</td>" for td in tds) + \
                        "</tr></table>"
            return fila_html, valor

    return None, None

# === Ejecutar para la URL de ejemplo ===
html = await fetch_html(URL)
fila_html, valor = extraer_num_empleados(html)

if fila_html:
    print("Fila encontrada (1.15.00.00):")
    display(HTML(fila_html))
    print("Valor N° de Empleados:", valor)
else:
    print("No se encontró la fila 1.15.00.00 en la Carátula.")


Fila encontrada (1.15.00.00):


1.15.00.00,N° de Empleados,6


Valor N° de Empleados: 6


In [ ]:
import pandas as pd

# Cargar archivo con tus URLs
df_urls = pd.read_excel("detalle_pj.xlsx")

resultados = []

for i, row in df_urls.iterrows():
    url = row["URL Ficha"]
    print(f"=== {i+1}/{len(df_urls)} ===")
    print("URL:", url)

    try:
        html = await fetch_html(url)
        fila_html, valor = extraer_num_empleados(html)
        resultados.append({
            "RUT": row.get("RUT"), 
            "Entidad": row.get("Entidad"),
            "URL": url,
            "N° Empleados": valor
        })
    except Exception as e:
        print("❌ Error en", url, ":", e)
        resultados.append({
            "RUT": row.get("RUT"),
            "Entidad": row.get("Entidad"),
            "URL": url,
            "N° Empleados": None
        })

# Convertir a DataFrame final
df_res = pd.DataFrame(resultados)
display(df_res.head())


=== 1/1979 ===
URL: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=3
=== 2/1979 ===
URL: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=3
=== 3/1979 ===
URL: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=3


Task exception was never retrieved
future: <Task finished name='Task-241' coro=<Connection.run.<locals>.init() done, defined at /opt/anaconda3/lib/python3.11/site-packages/playwright/_impl/_connection.py:307> exception=Exception('Connection.init: Connection closed while reading from the driver')>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/asyncio/tasks.py", line 277, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/playwright/_impl/_connection.py", line 308, in init
    self.playwright_future.set_result(await self._root_object.initialize())
                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/playwright/_impl/_connection.py", line 249, in initialize
    await self._channel.send(
  File "/opt/anaconda3/lib/python3.11/site-packages/playwright/_impl/_connection.py", line 69, in send
    return await self._connection.

❌ Error en https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=3 : Page.wait_for_selector: Target page, context or browser has been closed
=== 4/1979 ===
URL: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=3
❌ Error en https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=3 : Connection closed while reading from the driver
=== 5/1979 ===
URL: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=77672889&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOgAAf&control=svs&pestania=3
❌ Error en https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=77672889&grupo=&tipoentidad=CS

In [1]:
# --- CELDA 2 (script completo desde cero) ---
# Configuración
EXCEL_IN   = "detalle_pj.xlsx"               # archivo de entrada con las URLs
COL_URL    = "URL Ficha"                     # nombre EXACTO de la columna con la URL
CSV_OUT    = "num_empleados_caratula.csv"    # archivo de salida
PERIODO    = "12"                            # selector "Período"
ANIO       = "2024"                          # selector "Año"
HEADLESS   = True                            # poner False para ver el navegador
PAUSA_S    = 0.4                             # pausa entre requests (respeto al sitio)

# ---- Imports y setup ----
import re, time, pandas as pd, nest_asyncio, asyncio
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PWTimeout

nest_asyncio.apply()
pd.set_option("display.max_colwidth", None)

# ---- Navegación: abrir ficha (pestaña 3), setear período/año, apretar Consultar ----
async def fetch_html(url: str, periodo=PERIODO, anio=ANIO, headless=HEADLESS) -> str:
    browser = None
    context = None
    page = None
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=headless)
            context = await browser.new_context(locale="es-CL")
            page = await context.new_page()
            page.set_default_timeout(45000)

            await page.goto(url, wait_until="domcontentloaded")

            # Periodo/Año + "Consultar" (si existe el formulario)
            try:
                form = page.locator("form").first
                selects = form.locator("select")
                if await selects.count() >= 1:
                    await selects.nth(0).select_option(value=str(periodo))
                if await selects.count() >= 2:
                    try:
                        await selects.nth(1).select_option(value=str(anio))
                    except:
                        await selects.nth(1).select_option(label=str(anio))
                # Click en Consultar (probamos varias variantes)
                try:
                    await form.get_by_role("button", name="Consultar").click()
                except:
                    try:
                        await page.locator("button:has-text('Consultar')").click()
                    except:
                        await page.locator("input[type=submit]").first.click()
            except Exception:
                # Algunas fichas ya vienen con datos cargados; continuamos
                pass

            # Esperar a que exista la tabla de carátula y tenga filas
            try:
                await page.wait_for_selector("#caratula table", timeout=30000, state="visible")
                await page.wait_for_function(
                    "document.querySelectorAll('#caratula table tr').length > 5", timeout=30000
                )
            except PWTimeout:
                # Último recurso: esperar cualquier tabla visible
                await page.wait_for_selector("table", timeout=20000, state="visible")

            html = await page.content()
            return html
    finally:
        # Cierre ordenado (evita TargetClosedError colgantes)
        try:
            if page:    await page.close()
        except: pass
        try:
            if context: await context.close()
        except: pass
        try:
            if browser: await browser.close()
        except: pass

# ---- Parsing: extraer N° de Empleados desde div#caratula > table ----
def extraer_num_empleados(html: str):
    """
    Busca en div#caratula > table una fila cuyo primer <td> sea '1.15.00.00'
    y devuelve el valor del último <td>. Si no está, intenta por glosa 'N° de Empleados'.
    Retorna: (valor_str | None)
    """
    soup = BeautifulSoup(html, "lxml")
    tabla = soup.select_one("div#caratula table")
    if not tabla:
        return None

    # 1) exacto por código en la primera celda
    for tr in tabla.find_all("tr"):
        tds = tr.find_all("td", recursive=True)
        if not tds:
            continue
        first = re.sub(r"\s+", " ", tds[0].get_text(" ", strip=True))
        if first == "1.15.00.00":
            return tds[-1].get_text(" ", strip=True)  # conserva formato (ej: 6 o 10,00)

    # 2) fallback por glosa
    nodo = tabla.find(string=re.compile(r"N[\u00B0º]?\s*de\s*Empleados", re.I))
    if nodo:
        tr = nodo.find_parent("tr")
        if tr:
            tds = tr.find_all("td", recursive=True)
            if tds:
                return tds[-1].get_text(" ", strip=True)

    return None

# ---- Runner: recorre el Excel y guarda CSV ----
async def main_to_csv():
    # Leer Excel
    df_in = pd.read_excel(EXCEL_IN)
    if COL_URL not in df_in.columns:
        raise ValueError(f"No encontré la columna '{COL_URL}' en {EXCEL_IN}")

    resultados = []

    for i, row in df_in.iterrows():
        url = str(row[COL_URL]).strip() if pd.notna(row[COL_URL]) else ""
        if not url:
            continue

        print(f"\n=== {i+1}/{len(df_in)} ===")
        print("URL:", url)

        try:
            html = await fetch_html(url)
            valor = extraer_num_empleados(html)
            resultados.append({
                "URL": url,
                "N° de Empleados": valor
            })
            print("→ N° de Empleados:", valor)
        except Exception as e:
            print("❌ Error:", e)
            resultados.append({
                "URL": url,
                "N° de Empleados": None
            })

        time.sleep(PAUSA_S)  # pausa amable

    # Guardar CSV
    df_out = pd.DataFrame(resultados)
    df_out.to_csv(CSV_OUT, index=False, encoding="utf-8-sig")
    print(f"\n✅ CSV guardado: {CSV_OUT} (filas: {len(df_out)})")
    display(df_out.head())

# Ejecutar
await main_to_csv()



=== 1/1979 ===
URL: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=3
→ N° de Empleados: 6

=== 2/1979 ===
URL: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=3
→ N° de Empleados: 6

=== 3/1979 ===
URL: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=3
→ N° de Empleados: 6

=== 4/1979 ===
URL: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=3
→ N° de Empleados: 6

=== 5/1979 ===
URL: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=77672889&grupo=&tipoentidad=CSJUR&vig=

CancelledError: 